In [29]:
# No fast transitions
# The points close on the grid 
# should be close in real space
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

import meshplot
import matplotlib.pyplot as plt

from src.shared.sides import (
    sphered_vertices,
    to_vertices,
)
from src.shared.faces import (
    make_cube_faces,
)

from src.loss.distance_loss import get_distance_loss
from src.loss.angle_loss import get_angle_loss

torch.set_printoptions( precision=4, sci_mode=False)

In [2]:
device = torch.device('cuda')
n = 64
faces = make_cube_faces(n)
sphere = sphered_vertices(n, 0.5).to(device)
noise = torch.randn_like(sphere).to(device)

In [33]:
ratio = 0.01
noise_g = noise.clone() * ratio + sphere.clone() * (1 - ratio)
noise_g.requires_grad_(True)

opt = torch.optim.SGD([noise_g], lr=0.003)
print(get_angle_loss(noise_g))
print(get_distance_loss(noise_g))
meshplot.plot(to_vertices(noise_g).cpu().detach().numpy(), faces.numpy())

tensor(0.7956, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.5609, device='cuda:0', grad_fn=<MulBackward0>)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002714…

In [37]:
with torch.autograd.detect_anomaly():
    for step in range(10000):
        opt.zero_grad()
        
        d_loss = get_distance_loss(noise_g) 
        
        a_loss = get_angle_loss(noise_g)
        loss = d_loss + a_loss
        if step % 100 == 0:
            print(step, loss.item())      
        loss.backward()
        # Update parameters
        opt.step()

/home/bobi/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  """Entry point for launching an IPython kernel.


0 0.14438501000404358
100 0.12899015843868256
200 0.11966992914676666
300 0.11102961003780365
400 0.10295895487070084
500 0.09543734788894653
600 0.08837184309959412
700 0.08179635554552078
800 0.07575801014900208
900 0.07018434256315231
1000 0.06500596553087234
1100 0.060261864215135574
1200 0.055882927030324936
1300 0.051906146109104156
1400 0.04825578257441521
1500 0.0448855385184288
1600 0.041734252125024796
1700 0.03888119012117386
1800 0.03622274473309517
1900 0.033746637403964996
2000 0.031396884471178055
2100 0.02920241467654705
2200 0.02714626118540764
2300 0.02529820427298546
2400 0.02358693815767765
2500 0.021961156278848648
2600 0.02049505151808262
2700 0.019146030768752098
2800 0.01793137565255165
2900 0.016863731667399406
3000 0.015874426811933517
3100 0.014949310570955276
3200 0.014102540910243988
3300 0.013289885595440865
3400 0.012522643432021141
3500 0.01178640965372324
3600 0.011108518578112125
3700 0.010476394556462765
3800 0.009885337203741074
3900 0.00931741762906

In [36]:
meshplot.plot(to_vertices(noise_g).cpu().detach().numpy(), faces.numpy())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0014581…

In [35]:
import math
import torch
import torch.nn.functional as F

third = math.pi / 3

def get_angles(a, b, c, dim=1):
    ba = a - b
    bc = c - b    
    dot = (ba * bc).sum(dim=dim)
    norm = torch.linalg.norm(ba, dim=dim) * torch.linalg.norm(bc, dim=dim)
    cosine_angle = dot / (norm + 0.00001)
    #print(norm.min(), norm.max())
    #print(dot.min(), dot.max())
    #print(cosine_angle.min(), cosine_angle.max())
    return torch.acos(cosine_angle)# .clip(-1, 1)


def get_angle_loss(t, limit=third):
    a = t[:, :,  :-2, :]
    b = t[:, :, 1:-1, :]
    c = t[:, :, 2:,   :]
    ha = get_angles(a, b, c)
    ha = F.relu(math.pi - ha - limit)
    
    a = t[:, :, :,  :-2]
    b = t[:, :, :, 1:-1]
    c = t[:, :, :, 2:]
    va = get_angles(a, b, c)
    va = F.relu(math.pi - va - limit)    
    res = ha.mean() + va.mean()
    return res * 0.5

In [23]:
get_angle_loss(noise_g)

tensor(    0.0000, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.0023, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(    0.0000, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.0047, device='cuda:0', grad_fn=<MaxBackward1>)


tensor(0.1433, device='cuda:0', grad_fn=<MulBackward0>)